In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import datetime
import numpy as np
import pandas
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [3]:
data = pandas.read_csv('features.csv', index_col='match_id')
len(data)

97230

Подход 1: градиентный бустинг "в лоб"

In [4]:
data = pandas.read_csv('features.csv', index_col='match_id')
data.isna().sum().to_frame('nulls').sort_values(by='nulls', ascending=False).head(10)

,nulls
first_blood_player2,43987
radiant_flying_courier_time,27479
dire_flying_courier_time,26098
first_blood_time,19553
first_blood_player1,19553
first_blood_team,19553
dire_bottle_time,16143
radiant_bottle_time,15691
radiant_first_ward_time,1836
dire_first_ward_time,1826


1) Больше всего пропусков в признаках:
first_blood_player2 - 43987 пропусков, может объясняться тем, что к событию "первая кровь" часто причастен только один игрок, соответственно, признак для второго игрока часто будет пустым;
radiant_flying_courier_time	- 27479 пропусков, может объясняться тем, что команда в рассматриваемое время (первые 5 минут игры) не приобретает предмет "flying_courier"

Удалим признаки, связанные с итогом матча:

In [26]:
X_train = data.drop(
    ['duration', 'tower_status_radiant', 'tower_status_dire',
     'barracks_status_radiant', 'barracks_status_dire', 'radiant_win'], axis=1)
y_train = data.radiant_win

2) Целевая переменная - radiant_win

In [27]:
X_train = X_train.fillna(0)

cv = KFold(n_splits=5, shuffle=True, random_state=241)

In [28]:
def start_gradient_boosting(x_in):
    est_nums = [10, 20, 30, 50, 100, 250]
    means = []

    for estimators_number in est_nums:
        print('\nestimators_number =', estimators_number)

        clf = GradientBoostingClassifier(n_estimators=estimators_number, random_state=241)

        start_time = datetime.datetime.now()
        score = cross_val_score(clf, x_in, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
        finish_time = datetime.datetime.now() - start_time

        mean = np.mean(score)
        means.append(mean)

        print('score =', score)
        print('mean =', mean)
        print('time =', finish_time)

    return est_nums, means

In [29]:
nums, scores = start_gradient_boosting(X_train)


estimators_number = 10
score = [0.66943496 0.65627754 0.66390454 0.66281223 0.66950933]
mean = 0.6643877206345741
time = 0:00:25.624997

estimators_number = 20
score = [0.68939011 0.67567277 0.68201026 0.67977584 0.68741889]
mean = 0.6828535735340823
time = 0:00:41.587856

estimators_number = 30
score = [0.69641668 0.68365441 0.68731862 0.68728458 0.69280674]
mean = 0.6894962060591201
time = 0:01:01.577109

estimators_number = 50
score = [0.70382382 0.69325852 0.69452532 0.69565719 0.7000093 ]
mean = 0.6974548316948366
time = 0:01:41.969391

estimators_number = 100
score = [0.71211028 0.70275727 0.70340522 0.70450889 0.70884943]
mean = 0.7063262181631453
time = 0:03:24.605690

estimators_number = 250
score = [0.72079812 0.71236846 0.71331471 0.7139089  0.71780083]
mean = 0.7156382028679349
time = 0:08:29.129034


Для градиентного бустинга с 30-ю деревьями кросс-валидация проводилась 1 мин. 01 сек. 
При этом среднее значение метрики AUC-ROC составило ~0.69.
Имеет смысл использовать больше 30 деревьев, так как с увелечением количества деревьев увеличиваются значения метрики AUC-ROC. Однако, с увеличением числа деревьев, увеличивается и время выполнения бустинга. 
Чтобы ускорить обучение, можно:

1) использовать для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество, которое лучше всего брать случайным
2) уменьшить глубину деревьев в градиентом бустинге 

Подход 2: логистическая регрессия

In [32]:
def start_logistic_regression(x_in):
    powers = np.power(10.0, np.arange(-5, 6))
    means = []

    for c in powers:
        print('\nC =', c)

        clf = LogisticRegression(C=c, random_state=241, n_jobs=-1)

        start_time = datetime.datetime.now()
        score = cross_val_score(clf, x_in, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
        finish_time = datetime.datetime.now() - start_time

        mean = np.mean(score)
        means.append(mean)

        print('score =', score)
        print('mean =', mean)
        print('time =', finish_time)

    return powers, means

In [33]:
scaler = StandardScaler()

In [34]:
scale_x = scaler.fit_transform(X_train)

In [35]:
c_parameters, scores = start_logistic_regression(scale_x)


C = 1e-05
score = [0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
mean = 0.695120379847076
time = 0:00:06.609441

C = 0.0001
score = [0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
mean = 0.7112501143920594
time = 0:00:03.731386

C = 0.001
score = [0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
mean = 0.7161802463683579
time = 0:00:07.102149

C = 0.01
score = [0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
mean = 0.716341462186996
time = 0:00:07.897171

C = 0.1
score = [0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
mean = 0.7163100836533355
time = 0:00:09.314496

C = 1.0
score = [0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
mean = 0.716306583645544
time = 0:00:09.137148

C = 10.0
score = [0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
mean = 0.7163063399602339
time = 0:00:08.702426

C = 100.0
score = [0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
mean = 0.7163062657792336
time = 0:00:09.312202

C = 1000.0
scor

In [36]:
max(zip(scores, c_parameters))

(0.716341462186996, 0.01)

Лучшее значение метрики AUC-ROC = 0.716341462186996 при C = 0.01. Это выше чем при градиентном бустинге с 250 деревьявми.

Удалим категориальные признаки

In [37]:
def remove_category_features(x_in):
    del_list = ['{}{}_hero'.format(name, val) for val in range(1, 6) for name in ['r', 'd']]
    del_list.append('lobby_type')
    x = x_in.drop(del_list, axis=1)
    return x

In [38]:
x_without_category = remove_category_features(X_train)
scale_x = scaler.fit_transform(x_without_category)

In [39]:
c_parameters, scores = start_logistic_regression(scale_x)


C = 1e-05
score = [0.69961316 0.69150564 0.69419646 0.69425458 0.69571482]
mean = 0.6950569329910983
time = 0:00:02.456004

C = 0.0001
score = [0.71630684 0.70828717 0.70971489 0.71020968 0.71172337]
mean = 0.7112483906159718
time = 0:00:03.753200

C = 0.001
score = [0.72152794 0.71360976 0.71445497 0.71521698 0.71636831]
mean = 0.7162355910206267
time = 0:00:05.391951

C = 0.01
score = [0.7218164  0.71370124 0.71462628 0.71551614 0.71634469]
mean = 0.7164009506527343
time = 0:00:07.562794

C = 0.1
score = [0.72181205 0.7136378  0.7146095  0.71551012 0.71629945]
mean = 0.7163737844721111
time = 0:00:07.332341

C = 1.0
score = [0.72181039 0.71363045 0.71460884 0.71551016 0.71629392]
mean = 0.7163707526581122
time = 0:00:07.147561

C = 10.0
score = [0.72181043 0.71362965 0.71460893 0.71551033 0.71629306]
mean = 0.7163704793048005
time = 0:00:07.609673

C = 100.0
score = [0.72181067 0.71362945 0.71460889 0.71551027 0.71629322]
mean = 0.7163704962706652
time = 0:00:07.789792

C = 1000.0
s

In [40]:
max(zip(scores, c_parameters))

(0.7164009506527343, 0.01)

Удаление категориальных признаков повысило качество классификации. 
Максимальное значение AUC_ROC - 0.7164009506527343, что лучше, чем было без удаления категориальных признаков.
Данное значение оценка достигает так же при параметре регуляризации C = 0.01.

Количество героев в игре

In [47]:
heroes = pandas.read_csv('data/dictionaries/heroes.csv')

In [48]:
heroes.head()

,id,localized_name,name
0,1,Anti-Mage,antimage
1,2,Axe,axe
2,3,Bane,bane
3,4,Bloodseeker,bloodseeker
4,5,Crystal Maiden,crystal_maiden


In [49]:
heroes['name'].value_counts()

weaver                      1
razor                       1
visage                      1
venomancer                  1
juggernaut                  1
pugna                       1
chaos_knight                1
invoker                     1
tiny                        1
lich                        1
puck                        1
dark_seer                   1
rattletrap                  1
broodmother                 1
naga_siren                  1
ember_spirit                1
sven                        1
earthshaker                 1
obsidian_destroyer          1
skeleton_king               1
lone_druid                  1
medusa                      1
spectre                     1
tidehunter                  1
terrorblade                 1
clinkz                      1
riki                        1
vengefulspirit              1
enchantress                 1
faceless_void               1
                           ..
nevermore                   1
dragon_knight               1
spirit_bre

Всего в игре 112 героев

Формирование мешка слов по героям


In [50]:
def get_bow_data(in_data):
    x_pick = np.zeros((in_data.shape[0], 112))
    for i, match_id in enumerate(in_data.index):
        for p in range(5):
            x_pick[i, in_data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
            x_pick[i, in_data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
    bow_data = pandas.DataFrame(x_pick, index=in_data.index)
    return bow_data

In [51]:
bow_train_data = get_bow_data(data)
final_x = pandas.concat([x_without_category, bow_train_data], axis=1)
scale_x = scaler.fit_transform(final_x)

In [52]:
c_parameters, scores = start_logistic_regression(scale_x)


C = 1e-05
score = [0.71907467 0.71117649 0.71512939 0.71288168 0.71586231]
mean = 0.7148249064297479
time = 0:00:10.001422

C = 0.0001
score = [0.74685672 0.74006714 0.74277168 0.74026225 0.74423779]
mean = 0.7428391165802382
time = 0:00:06.369158

C = 0.001
score = [0.75515333 0.74931124 0.75181648 0.74901026 0.75305284]
mean = 0.751668830891463
time = 0:00:10.594970

C = 0.01
score = [0.75521596 0.74985583 0.75218088 0.74951097 0.75308879]
mean = 0.7519704890317176
time = 0:00:14.126786

C = 0.1
score = [0.75513027 0.74989157 0.75213472 0.74953537 0.75293117]
mean = 0.7519246209167468
time = 0:00:14.612443

C = 1.0
score = [0.75511969 0.74989604 0.75212742 0.74953884 0.75290574]
mean = 0.7519175479357812
time = 0:00:15.954828

C = 10.0
score = [0.75511883 0.74989652 0.75212657 0.74953969 0.75290278]
mean = 0.7519168780227994
time = 0:00:15.397924

C = 100.0
score = [0.75511879 0.74989678 0.75212649 0.74953983 0.75290249]
mean = 0.7519168758411852
time = 0:00:15.651711

C = 1000.0
sc

In [53]:
max(zip(scores, c_parameters))

(0.7519704890317176, 0.01)

При добавлении "мешка слов" по героям значение оценки качесва классификации значительно улучшилось и составило 0.7519704890317176. Это связано с тем, что теперь категориальные признаки используются более разумно, т. к. герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Наилучшего качества логистическая регрессия с мешком слов по героям достигает при параметре регуляризации C = 0.01.


Проверка модели на тестовых данных

In [62]:
x_test = pandas.read_csv('features_test.csv', index_col='match_id')
x_test = x_test.fillna(0)
bow_test_data = get_bow_data(x_test)
x_test_without_category = remove_category_features(x_test)
final_x_test = pandas.concat([x_test_without_category, bow_test_data], axis=1)
scale_x_test = scaler.fit_transform(final_x_test)

In [67]:
model = LogisticRegression(C=0.01, random_state=241)

model.fit(scale_x, y_train)
y_test = model.predict_proba(scale_x_test)
p = list(map(lambda v: max(v), y_test))
max_p = max(p)
min_p = min(p)
print('max p =', max_p)
print('min p =', min_p)

max p = 0.996328715925428
min p = 0.5000078481322778
